**Avant de lancer ce notebook merci de générer les csv à l'aide du [notebook suivant](Create-csv-files.ipynb).**

In [1]:
# Commenter la configuration non utilisée

# Configuration production
mongoDBUrl = 'mongodb017.westeurope.cloudapp.azure.com'
port = 32002

# Configuration dev
#mongoDBUrl = 'localhost'
#port = 27017

In [2]:
import pandas as pd
import numpy as np
import json
from pymongo import MongoClient

In [3]:
client = MongoClient( mongoDBUrl, port )
db = client['db_credit']

In [4]:
def drop_collections(client, db):
    
    for collection_name in db.list_collection_names():
        collection = db[collection_name]
        collection.drop()

In [5]:
drop_collections(client, db)

Dénormalisation retenue:

In [6]:
{
    "members":
    {
        "member_no": "1",
        "lastname": "DOUGLAS",
        "firstname": "Michael",
        "phone_number":"blabla",

        "location":
        {
            "country":"France",
            "region_no":"IDF",
            "city":"Paris",
            "mail_code":"75000",
            "street":"blabla"
        },

        "balance":
        {
            "prev_balance": "1000",
            "curr_balance": "1200"
        },

        "_comment":"No need to have a list, each employee have only 1 corporation",
        "corporation":
        {
            "corp_no":1,
            "corp_name":"blabla",
            "region_no":"XX"
        },

        "charges": [
            {
                "charge_no": "125",
                "category_no": "12",
                "category_desc": "Travel",
                "charge_dt": "2000-02-22 10:43:38",
                "charge_amount": "336.0000",

                "provider_no":12,
                "provider_name":"BNP Paribas",
                "provider_phone_number": "blabla",
                "payments":[1546, 1547,... , 1578]
            }
        ]
    },
     "providers":
        {
            "provider_no":1,
            "provider_name":"BNP",
            "location":
            {
                "country":"France",
                "region_no":"IDF",
                "city":"Paris",
                "mail_code":"75000"
            }
        },
 
 
    "payments":
        {
            "payment_no":1546,
            "payment_dt":"2000-02-22 10:43:38",
            "payment_amt":125.00,
            "payment_principal": 100,
            "payment_interest":25,
        }
}


{'members': {'member_no': '1',
  'lastname': 'DOUGLAS',
  'firstname': 'Michael',
  'phone_number': 'blabla',
  'location': {'country': 'France',
   'region_no': 'IDF',
   'city': 'Paris',
   'mail_code': '75000',
   'street': 'blabla'},
  'balance': {'prev_balance': '1000', 'curr_balance': '1200'},
  '_comment': 'No need to have a list, each employee have only 1 corporation',
  'corporation': {'corp_no': 1, 'corp_name': 'blabla', 'region_no': 'XX'},
  'charges': [{'charge_no': '125',
    'category_no': '12',
    'category_desc': 'Travel',
    'charge_dt': '2000-02-22 10:43:38',
    'charge_amount': '336.0000',
    'provider_no': 12,
    'provider_name': 'BNP Paribas',
    'provider_phone_number': 'blabla',
    'payments': [1546, 1547, Ellipsis, 1578]}]},
 'providers': {'provider_no': 1,
  'provider_name': 'BNP',
  'location': {'country': 'France',
   'region_no': 'IDF',
   'city': 'Paris',
   'mail_code': '75000'}},
 'payments': {'payment_no': 1546,
  'payment_dt': '2000-02-22 10:43:3

In [7]:
Members = pd.read_csv('Members.csv')
Corporations = pd.read_csv('Corporations.csv')
Categories = pd.read_csv('Categories.csv')
Providers = pd.read_csv('Prodivers.csv')
Charges = pd.read_csv('Charges.csv')
Statements = pd.read_csv('Statements.csv')
Payments = pd.read_csv('Payments.csv')

In [8]:
Charges

,Charge type,Charge amount,Duration,dt,member_no,provider_no,charge_no
0,1,500000,48,2020-04-24,87,63,0
1,0,46318,36,2020-09-02,9839,16,1
2,2,3876,48,2021-11-08,1541,92,2
3,1,777031,36,2020-01-28,8703,131,3
4,0,139127,24,2021-12-06,1915,86,4
...,...,...,...,...,...,...,...
49995,1,1492652,60,2020-09-11,760,105,49995
49996,2,13202,12,2021-12-01,7400,74,49996
49997,0,284619,36,2020-04-27,5210,35,49997
49998,0,4212,36,2021-12-09,3030,156,49998


In [9]:
Payments

,payment_principal,payment_interest,due_dt,charge_no,member_no,payment_no,payment_amt
0,10416.666667,979.166667,2020-05-24,0,87,0,11395.833333
1,10416.666667,958.333333,2020-06-24,0,87,1,11375.000000
2,10416.666667,937.500000,2020-07-24,0,87,2,11354.166667
3,10416.666667,916.666667,2020-08-24,0,87,3,11333.333333
4,21584.194444,1510.893611,2020-02-28,3,8703,132,23095.088056
...,...,...,...,...,...,...,...
58323,5964.937500,489.124875,2020-08-06,49993,8896,1798410,6454.062375
58324,7906.083333,553.425833,2020-05-27,49997,5210,1798548,8459.509167
58325,7906.083333,537.613667,2020-06-27,49997,5210,1798549,8443.697000
58326,7906.083333,521.801500,2020-07-27,49997,5210,1798550,8427.884833


In [10]:
member_no = 907

charges = Charges[Charges.member_no == member_no]
charges

,Charge type,Charge amount,Duration,dt,member_no,provider_no,charge_no
14602,0,354020,60,2021-12-20,907,34,14602
15673,0,125000,60,2021-02-15,907,30,15673
40810,1,500000,24,2021-01-30,907,40,40810


In [11]:
providers_no = charges.provider_no.values
providers_no

array([34, 30, 40])

In [12]:
providers = []
for provider_no in providers_no:
    provider = Providers[Providers.provider_no == provider_no].loc[provider_no]
    providers.append(provider)
providers

[provider_no                       34
 provider_name    JPMorgan Chase & Co
 phone_number          (975) 682-1923
 country                United States
 region_no                         KY
 city                      Louisville
 mail_code                      40203
 street           128 East Oak Street
 Name: 34, dtype: object,
 provider_no                         30
 provider_name                Citigroup
 phone_number            (467) 729-1588
 country                  United States
 region_no                           AZ
 city                          Glendale
 mail_code                        85302
 street           9110 North 63rd Drive
 Name: 30, dtype: object,
 provider_no                          40
 provider_name    PNC Financial Services
 phone_number             (542) 040-0499
 country                   United States
 region_no                            GA
 city                           Savannah
 mail_code                         31404
 street                   301 East Co

In [13]:
charges_no = charges.charge_no.values
charges_no

array([14602, 15673, 40810])

In [14]:
payments = []
for charge_no in charges_no:
    payments_no = Payments[Payments.charge_no == charge_no]['payment_no'].values
    payments.append(list(payments_no))
payments

[[], [], []]

In [15]:
charges_json = []
for i in range(len(charges_no)):
    charge = charges.iloc[i-1]
    provider = providers[i]
    payments_list = payments[i]
    charge_json = {
        "charge_no": charge.charge_no,
        "category_no": charge['Charge type'],
        #"category_desc": "Travel",
        "charge_dt": charge['dt'],
        "charge_amount": charge['Charge amount'],

        "provider_no": provider.provider_no,
        "provider_name": provider.provider_name,
        "provider_phone_number": provider.phone_number,
        "payments": payments_list
    }
    charges_json.append(charge_json)
    
charges_json

[{'charge_no': 40810,
  'category_no': 1,
  'charge_dt': '2021-01-30',
  'charge_amount': 500000,
  'provider_no': 34,
  'provider_name': 'JPMorgan Chase & Co',
  'provider_phone_number': '(975) 682-1923',
  'payments': []},
 {'charge_no': 14602,
  'category_no': 0,
  'charge_dt': '2021-12-20',
  'charge_amount': 354020,
  'provider_no': 30,
  'provider_name': 'Citigroup',
  'provider_phone_number': '(467) 729-1588',
  'payments': []},
 {'charge_no': 15673,
  'category_no': 0,
  'charge_dt': '2021-02-15',
  'charge_amount': 125000,
  'provider_no': 40,
  'provider_name': 'PNC Financial Services',
  'provider_phone_number': '(542) 040-0499',
  'payments': []}]

In [16]:
member = Members.loc[lambda x: x.member_no == member_no].loc[member_no]
member

member_no                     907
lastname                   Wright
firstname                  Thomas
phone_number       (764) 977-6331
country             United States
region_no                      TN
city                    Nashville
mail_code                   37211
street          510 Leeanne Drive
corp_no                       301
prev_balance           580.262863
curr_balance          1704.064073
Name: 907, dtype: object

In [17]:
corporation = Corporations.loc[lambda x: x.corp_no == member.corp_no].loc[member.corp_no]
corporation

corp_no                            301
corp_name               Skinner-Deleon
phone_number            (154) 346-4621
country                  United States
region_no                           MD
city                         Edgewater
mail_code                        21037
street          105 Bear Creek Parkway
Name: 301, dtype: object

In [18]:
member_json = {
    "member_no": member_no,
    "lastname": member.lastname,
    "firstname": member.firstname,
    "phone_number": member.phone_number,

    "location":
    {
        "country": member.country,
        "region_no": member.region_no,
        "city": member.city,
        "mail_code": member.mail_code,
        "street": member.street
    },

    "balance":
    {
        "prev_balance": member.prev_balance,
        "curr_balance": member.curr_balance
    },
    "corporation":
    {
        "corp_no": corporation.corp_no,
        "corp_name": corporation.corp_name,
        "region_no": corporation.region_no
    },

    "charges": charges_json
}
member_json

{'member_no': 907,
 'lastname': 'Wright',
 'firstname': 'Thomas',
 'phone_number': '(764) 977-6331',
 'location': {'country': 'United States',
  'region_no': 'TN',
  'city': 'Nashville',
  'mail_code': 37211,
  'street': '510 Leeanne Drive'},
 'balance': {'prev_balance': 580.2628631213131,
  'curr_balance': 1704.064073211901},
 'corporation': {'corp_no': 301,
  'corp_name': 'Skinner-Deleon',
  'region_no': 'MD'},
 'charges': [{'charge_no': 40810,
   'category_no': 1,
   'charge_dt': '2021-01-30',
   'charge_amount': 500000,
   'provider_no': 34,
   'provider_name': 'JPMorgan Chase & Co',
   'provider_phone_number': '(975) 682-1923',
   'payments': []},
  {'charge_no': 14602,
   'category_no': 0,
   'charge_dt': '2021-12-20',
   'charge_amount': 354020,
   'provider_no': 30,
   'provider_name': 'Citigroup',
   'provider_phone_number': '(467) 729-1588',
   'payments': []},
  {'charge_no': 15673,
   'category_no': 0,
   'charge_dt': '2021-02-15',
   'charge_amount': 125000,
   'provider_n

In [19]:
P = []

In [20]:
def get_charges(member_no):
    charges = Charges[Charges.member_no == member_no]
    return charges

def get_member(member_no):
    member = Members.loc[lambda x: x.member_no == member_no].loc[member_no]
    return member

def get_providers(charges):
    providers_no = charges.provider_no.values
    providers = []
    for provider_no in providers_no:
        provider = Providers[Providers.provider_no == provider_no].loc[provider_no]
        providers.append(provider)
    return providers

def get_payments(charges):
    charges_no = charges.charge_no.values
    payments = []
    for charge_no in charges_no:
        payments_no = Payments[Payments.charge_no == charge_no]['payment_no'].values
        payments.append(list(payments_no))
    return payments

def get_charges_json(member_no):
    charges = get_charges(member_no)
    providers = get_providers(charges)
    payments = get_payments(charges)
    
    charges_json = []
    for i in range(len(charges)):
        charge = charges.iloc[i-1]
        provider = providers[i]
        payments_list = payments[i]
        json = {
            "charge_no": charge.charge_no,
            "category_no": charge['Charge type'],
            "category_desc": get_category_name(charge['Charge type']),
            "charge_dt": charge['dt'],
            "charge_amount": charge['Charge amount'],

            "provider_no": provider.provider_no,
            "provider_name": provider.provider_name,
            "provider_phone_number": provider.phone_number,
            "provider_city": provider.city,
            "payments": payments_list
        }
        charges_json.append(json)

    return charges_json

def get_category_name(category_no):
    return Categories.loc[lambda x: x.category_no == category_no].category_name.loc[category_no]

def get_corporation(member):
    corporation = Corporations.loc[lambda x: x.corp_no == member.corp_no].loc[member.corp_no]
    return corporation

def get_member_json(member_no):
    charges_json = get_charges_json(member_no)
    member = get_member(member_no)
    corporation = get_corporation(member)
    
    member_json = {
        "member_no": member_no,
        "lastname": member.lastname,
        "firstname": member.firstname,
        "phone_number": member.phone_number,

        "location":
        {
            "country": member.country,
            "region_no": member.region_no,
            "city": member.city,
            "mail_code": member.mail_code,
            "street": member.street
        },
        
        "balance":
        {
            "prev_balance": member.prev_balance,
            "curr_balance": member.curr_balance
        },

        
        "corporation":
        {
            "corp_no": corporation.corp_no,
            "corp_name": corporation.corp_name,
            "region_no": corporation.region_no
        },

        "charges": charges_json
    }
    return member_json

def get_members_json():
    return Members.member_no.apply(lambda x: get_member_json(x)).to_list()
    # On utilise .uniques() pour éviter les doublons

In [21]:
def get_providers_json():
    providers = [
        {
            "provider_no": provider.provider_no,
            "provider_name": provider.provider_name,
            "location":
            {
                "country": provider.country,
                "region_no": provider.region_no,
                "city": provider.city,
                "mail_code": provider.mail_code
            }
        } for (_,provider) in Providers.iterrows()]
    return providers

def get_payments_json():
    payments = [{
            "payment_no": payment.payment_no,
            "payment_dt": payment.due_dt,
            "payment_amt": payment.payment_amt,
            "payment_principal": payment.payment_principal,
            "payment_interest": payment.payment_interest,
        } for (_,payment) in Payments.iterrows()]
    return payments

def final_json():
    return {
        'members': get_members_json(),
        'providers': get_providers_json(),
        'payments': get_payments_json()
    }

# Class pour résoudre les problèmes d'encoding entre numpy et le module json
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

def _export_mongo(data, filename, collection_name):
    with open(filename, 'w') as outfile:
        json.dump(data, outfile, cls=NpEncoder, indent=4)

    client = MongoClient( mongoDBUrl, port )
    db = client['db_credit']
    collection = db[collection_name]
    #indexName = collection_name.replace('s','')+'_no'
    #print(collection)
    #print(indexName)
    #collection.createIndex({indexName:1})

    with open(filename) as f:
        file_data = json.load(f)
        collection.insert_many(file_data)
        client.close()
        
def members_export_mongo():
    data = get_members_json()
    filename = 'members.json'
    collection_name = 'members'
    _export_mongo(data, filename, collection_name)
    
def providers_export_mongo():
    data = get_providers_json()
    filename = 'providers.json'
    collection_name = 'providers'
    _export_mongo(data, filename, collection_name)
    
def payments_export_mongo():
    data = get_payments_json()
    filename = 'payments.json'
    collection_name = 'payments'
    _export_mongo(data, filename, collection_name)
    
def export_mongo():
    members_export_mongo()
    providers_export_mongo()
    payments_export_mongo()

**À cette étape on va créer les index directement en ligne de commande**

In [28]:
export_mongo()

In [29]:
client = MongoClient( mongoDBUrl, port )
db = client['db_credit']
db.list_collection_names()

['payments', 'providers', 'members']

In [30]:
collection = db['members']
collection.count_documents({})

10000

In [31]:
collection = db['payments']
collection.count_documents({})

58328

In [32]:
collection = db['providers']
collection.count_documents({})

200

In [33]:
client.close()